In [ ]:
#= Importy i załadowanie danych =#
using JLD2
using Printf
using Statistics
using LinearAlgebra

include("src/AD.jl")
include("src/NN.jl")

Main.NN

In [ ]:
#= Przygotowanie danych =#
file = load("data/imdb_dataset_prepared.jld2")
X_train = Int.(file["X_train"])
y_train = vec(Float32.(file["y_train"]))
X_test  = Int.(file["X_test"])
y_test  = vec(Float32.(file["y_test"]))
embeddings = file["embeddings"]
vocab = file["vocab"]
embedding_dim = size(embeddings, 1)

50

In [73]:
dataset = NN.DataLoader((X_train, y_train), batchsize=64, shuffle=true)

Main.NN.DataLoader((Float32[6391.0 286.0 … 11382.0 8311.0; 143.0 286.0 … 4254.0 2891.0; … ; 12849.0 12849.0 … 12849.0 12849.0; 12849.0 12849.0 … 12849.0 12849.0], Float32[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]), 64, true, [19117, 37079, 10136, 6971, 37305, 34266, 4407, 37609, 12832, 36333  …  33552, 28557, 36327, 19713, 11269, 9815, 3502, 14671, 19805, 33710])

In [74]:
model = NN.Chain(
    x -> Int.(x),
    NN.Embedding(length(vocab), embedding_dim, embeddings),
    x -> permutedims(x, (2, 1, 3)),
    NN.Conv1D(3, embedding_dim, 8, NN.relu),
    NN.MaxPool1D(8),
    NN.flatten,
    NN.Dense(128, 1, NN.sigmoid)
)

Main.NN.Chain(Any[var"#61#63"(), Main.NN.Embedding(12849, 50, Float32[0.90951 -0.58014 … 0.014323 0.0; -0.20702 -1.1316 … -0.74624 0.0; … ; -0.60396 0.1722 … -0.079873 0.0; -0.31136 1.2622 … -0.89164 0.0], Int64[]), var"#62#64"(), Main.NN.Conv1D(3, 50, 8, Float32[-0.16570108 0.027323633 … -0.015469636 0.0012371079; 0.13452297 -0.026739301 … 0.09288249 -0.25757426; -0.05129965 0.06483751 … 0.05390835 -0.024678327;;; -0.03966974 -0.03416187 … 0.047850925 0.06195336; 0.04495138 -0.13327932 … -0.084138215 0.017433368; 0.017740173 0.10842881 … 0.13109474 -0.06647755;;; 0.0563288 -0.029122261 … 0.09456226 0.15208714; 0.046646964 0.202473 … 0.0038042006 0.16045892; 0.12018149 -0.09307607 … -0.08516864 0.317648;;; -0.06207592 -0.060179364 … 0.10968941 0.0028341427; -0.098544136 -0.18816434 … -0.13574745 0.040246736; -0.007876265 0.0655038 … 0.06354014 0.04605036;;; 0.1200056 -0.11073338 … -0.18567623 -0.003948292; 0.0340111 -0.019381054 … -0.034958813 -0.06966194; -0.026758011 0.03950359 … -0.

In [75]:
accuracy(m, x, y) = mean((vec(m(x)) .> 0.5) .== (y .> 0.5))

accuracy (generic function with 1 method)

In [ ]:
#= Trening modelu =#
opt = NN.Adam(Float32(0.001))
epochs = 5
NN.train_model(model, dataset, X_test, y_test, opt, epochs)

Epoch: 1 (20.43s) 	Train: (l: 0.63, a: 0.66) 	Test: (l: 0.52, a: 0.75) 	Memory allocated: 25381.363 MB
Epoch: 2 (20.22s) 	Train: (l: 0.48, a: 0.77) 	Test: (l: 0.46, a: 0.79) 	Memory allocated: 25369.554 MB
Epoch: 3 (20.37s) 	Train: (l: 0.45, a: 0.79) 	Test: (l: 0.44, a: 0.81) 	Memory allocated: 25377.553 MB
Epoch: 4 (20.20s) 	Train: (l: 0.43, a: 0.80) 	Test: (l: 0.42, a: 0.81) 	Memory allocated: 25381.875 MB
Epoch: 5 (20.34s) 	Train: (l: 0.41, a: 0.82) 	Test: (l: 0.41, a: 0.82) 	Memory allocated: 25384.939 MB
